<a href="https://colab.research.google.com/github/kingbingodbin/NLP_models/blob/main/XLnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
from torchtext.data.utils import get_tokenizer
from torchtext.data import Field
import pandas as pd
import os

In [2]:
import torchtext
import io

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import os
#os.listdir('drive/My Drive/datasets/wikitext-2')

In [11]:
#install package for subword tokenizer
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 2.7MB/s 


tokenizer

In [13]:
import sentencepiece as spm

In [15]:
import urllib.request
import io
import sentencepiece as spm

# Loads model from URL as iterator and stores the model to BytesIO.
model = io.BytesIO()
with urllib.request.urlopen(
    'https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt'
) as response:
  spm.SentencePieceTrainer.train(
      sentence_iterator=response, model_writer=model, vocab_size=1000)

# Serialize the model as file.
# with open('out.model', 'wb') as f:
#   f.write(model.getvalue())

# Directly load the model from serialized model.
sp = spm.SentencePieceProcessor(model_proto=model.getvalue())
print(sp.encode('this is test'))

[64, 47, 4, 13, 391]


In [16]:
aa = torchtext.datasets.language_modeling.WikiText2.splits()

NameError: ignored

In [32]:
dataset_base_path = 'drive/My Drive/datasets/'
path = dataset_base_path + 'wikitext-2/'
#use reversable tokenizer for flexible 
TEXT = torchtext.data.ReversibleField(tokenize = get_tokenizer("revtok"),
                            init_token = '<sos>',
                            eos_token = '<eos>',
                            sequential=True,
                            lower=True,
                            include_lengths=True)

In [38]:
dtype = torch.float
device = torch.device("cpu")

In [39]:
tr_dat,val_dat,te_dat = torchtext.datasets.language_modeling.WikiText2.splits(TEXT)

In [40]:
[tr_dat.examples[0].text][0][:10]

[' ',
 ' ',
 '<eos>',
 ' ',
 ' = ',
 ' valkyria ',
 ' chronicles ',
 ' iii ',
 ' = ',
 ' ']

In [41]:
[val_dat.examples[0].text][0][:10]

[' ', ' ', '<eos>', ' ', ' = ', ' homarus ', ' gammarus ', ' = ', ' ', '<eos>']

In [42]:
[te_dat.examples[0].text][0][:10]

[' ', ' ', '<eos>', ' ', ' = ', ' robert ', ' <', ' unk ', '> ', ' = ']

In [43]:
#seperating wikitext2 data to documents
def sep_to_docs(data : list, doc_sep : str, token_sep : str) -> list:
  """returns documents from split input text chunk
  """
  result = []
  docs = (''.join(data)).split(doc_sep)
  for i in docs:
    curr_doc = (''.join(i)).split(token_sep)
    curr_doc = [i for i in curr_doc if i != '']
    if len(curr_doc) >= 1:
      result.append(curr_doc)
  return result

In [44]:
tr_docs = sep_to_docs(data = [tr_dat.examples[0].text][0], doc_sep = '<eos>  =', token_sep = '  ')
val_docs = sep_to_docs(data = [val_dat.examples[0].text][0], doc_sep = '<eos>  =', token_sep = '  ')
te_docs = sep_to_docs(data = [te_dat.examples[0].text][0], doc_sep = '<eos>  =', token_sep = '  ')

In [45]:
print(len(tr_docs),len(val_docs),len(te_docs))

6211 620 708


In [ ]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed

In [46]:
#building vocab
TEXT.build_vocab(sum(tr_docs, []))

In [59]:
tr_docs[1]

['=',
 'gameplay',
 '=',
 '=',
 '<eos>',
 '<eos>',
 'as',
 'with',
 'previous',
 '< unk >',
 'chronicles',
 'games',
 ',',
 'valkyria',
 'chronicles',
 'iii',
 'is',
 'a',
 'tactical',
 'role',
 '@ - @',
 'playing',
 'game',
 'where',
 'players',
 'take',
 'control',
 'of',
 'a',
 'military',
 'unit',
 'and',
 'take',
 'part',
 'in',
 'missions',
 'against',
 'enemy',
 'forces',
 '.',
 'stories',
 'are',
 'told',
 'through',
 'comic',
 'book',
 '@ - @',
 'like',
 'panels',
 'with',
 'animated',
 'character',
 'portraits',
 ',',
 'with',
 'characters',
 'speaking',
 'partially',
 'through',
 'voiced',
 'speech',
 'bubbles',
 'and',
 'partially',
 'through',
 '< unk >',
 'text',
 '.',
 'the',
 'player',
 'progresses',
 'through',
 'a',
 'series',
 'of',
 'linear',
 'missions',
 ',',
 'gradually',
 'unlocked',
 'as',
 'maps',
 'that',
 'can',
 'be',
 'freely',
 '< unk >',
 'through',
 'and',
 'replayed',
 'as',
 'they',
 'are',
 'unlocked',
 '.',
 'the',
 'route',
 'to',
 'each',
 'story'

In [60]:
TEXT.numericalize((tr_docs[:2],[len(tr_docs[0]),len(tr_docs[1])]))

ValueError: ignored

In [63]:
tr_docs[:2]

[['valkyria',
  'chronicles',
  'iii',
  '=',
  '<eos>',
  '<eos>',
  'senjō',
  'no',
  'valkyria',
  '3',
  ':',
  '< unk >',
  'chronicles',
  '(',
  'japanese',
  ':',
  '戦場のヴァルキュリア 3',
  ',',
  'lit',
  '.',
  'valkyria',
  'of',
  'the',
  'battlefield',
  '3',
  ')',
  ',',
  'commonly',
  'referred',
  'to',
  'as',
  'valkyria',
  'chronicles',
  'iii',
  'outside',
  'japan',
  ',',
  'is',
  'a',
  'tactical',
  'role',
  '@ - @',
  'playing',
  'video',
  'game',
  'developed',
  'by',
  'sega',
  'and',
  'media . vision',
  'for',
  'the',
  'playstation',
  'portable',
  '.',
  'released',
  'in',
  'january',
  '2011',
  'in',
  'japan',
  ',',
  'it',
  'is',
  'the',
  'third',
  'game',
  'in',
  'the',
  'valkyria',
  'series',
  '.',
  '< unk >',
  'the',
  'same',
  'fusion',
  'of',
  'tactical',
  'and',
  'real',
  '@ - @',
  'time',
  'gameplay',
  'as',
  'its',
  'predecessors',
  ',',
  'the',
  'story',
  'runs',
  'parallel',
  'to',
  'the',
  'first',
 

In [ ]:


class XLnet_Input(torch.utils.data.Dataset):


  def __init__(self, input_dat: list, word2index: dictionary, index2word: dictionary, segment_length: int, seq_length: int, ):

    #
    #dict
    #context_encoding, segment_encoding
    #positional encoding

    
    self.dat_shape = data.shape
    self.root_dir = root_dir
    self.segment_len = segment_len
    self.word_dict = False
    self.data = data

    if do_shuffle:
      shuffle()
    
    segmentation()

  def segmentation(self):
    self.segments = 
  
  

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    doc_paths = os.path.join(self.root_dir, self.target_csv.iloc[idx, 0])
    for doc_path in doc_paths
    doc_file = open(doc_path, 'r')

    lines = doc_file.readlines()

    if word_dict:
      for line in lines:
        for 

    


    doc_file.close()


    f = open("C:/doit/새파일.txt", 'r')


      landmarks = self.landmarks_frame.iloc[idx, 1:]
      landmarks = np.array([landmarks])
      landmarks = landmarks.astype('float').reshape(-1, 2)
      sample = {'image': image, 'landmarks': landmarks}

    return sample

  def __len__(self):
    return len(self.input)

In [ ]:
self.target_csv = pd.read_csv(traget_csv)
    self.dat_shape = data.shape
    self.root_dir = root_dir
    self.segment_len = segment_len
    self.word_dict = False
    self.data = data

    if do_shuffle:
      shuffle()
    
    segmentation()

  def segmentation(self):
    self.segments = 

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    doc_paths = os.path.join(self.root_dir, self.target_csv.iloc[idx, 0])
    for doc_path in doc_paths
    doc_file = open(doc_path, 'r')

    lines = doc_file.readlines()

    if word_dict:
      for line in lines:
        for 

    


    doc_file.close()


    f = open("C:/doit/새파일.txt", 'r')


      landmarks = self.landmarks_frame.iloc[idx, 1:]
      landmarks = np.array([landmarks])
      landmarks = landmarks.astype('float').reshape(-1, 2)
      sample = {'image': image, 'landmarks': landmarks}

    return sample

  def __len__(self):
    return len(self.data)

'<eos>  ='

In [ ]:
aa = [tr_dat.examples[0].text][0][:1000]

In [ ]:
result = []
docs = (''.join(aa)).split('<eos>  =')
for i in temp:
  curr_doc = (''.join(i)).split('  ')
  curr_doc = [i for i in curr_doc if i != '']
  result.append(curr_doc)

In [ ]:
train_data[0]

tensor([    7,    29,  4031,  2997,   447,  3285,     4,   133,   444,    36,
         3745,     3,   838,     6, 24073,  1484,     4,    25, 13603,    71])

In [ ]:
train_data[1]

tensor([    7,  1781,    20,   179, 15130,   765,   632,    59,   490,  6997,
           27,     7,     6,     4,   480,  5051,    60,    15,     8,     9])

<bound method Field.build_vocab of <torchtext.data.field.Field object at 0x7f1ca121d400>>

In [ ]:
en_textfield.build_vocab(tr_dat, min_freq = 2)

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (tr_dat, val_dat, test_dat),
    batch_size = BATCH_SIZE,
    device = device)

In [ ]:
bb

(tensor([[   9,   11, 3875,  ...,    4,    9,    9]]), tensor([2088628]))

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
ds = tfds.load('glue', split='train', as_supervised=False, shuffle_files=False)

In [ ]:
for i in ds:
  print(i)

NameError: ignored

In [ ]:
aa.load()

AttributeError: ignored

In [ ]:
np.array([1,2,3]).shape()

TypeError: ignored

In [ ]:
ddd = aa(a = np.array([1,2,4,5]),b=np.array([1,2,4,5]))

In [ ]:
exec('b=ddd.a.sum()')

In [ ]:
len(np.array([1,2,3]))

3

In [ ]:
sample_dict.get('aa')

12

In [ ]:
sample_dict = {'aa': 12, 'bb':13}

In [ ]:
'list is list'.split(' ')

['list', 'is', 'list']

In [ ]:
dtype = torch.float
device = torch.device("cpu")

In [ ]:
#활용된 잡지식

#pytorch dataset 불러오기 
#https://pytorch.org/text/datasets.html

#자동으로 추가해야할 내용들
#To avoid clutter, we omit the implementation details including multi-head attention, residual connection, layer normalization and position-wise feed-forward as used in Transformer(-XL). The details are included in Appendix A.2 for reference

#데이터 로딩
#data loading
# https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html

#함수 오버라이딩 방법
#super(MyModel, self).__init__()

#einstein summation
#https://rockt.github.io/2018/04/30/einsum

#defining method with string
#https://stackoverflow.com/questions/11553721/using-a-string-variable-as-a-variable-name

#kwargs,args
#https://brunch.co.kr/@princox/180

In [ ]:
# Two-stream attention cell
#input : q_size, k_size, v_size, segment_index, attention_size, heads =8, **kwargs = {init_Qstream_q_LT_weight, init_Qstream_q_K_weight,init_Qstream_q_R_weight, init_Qstream_k_E_LT_weight, init_Qstream_k_R_LT_weight, init_Qstream_v_LT_weight,
#                                                                            init_Qstream_segment_same_weight, init_Qstream_segment_notsame_weight, init_Qstream_segment_bias, init_Qstream_output_weight,
#                                                                            init_Cstream_q_LT_weight, init_Cstream_q_K_weight,init_Cstream_q_R_weight, init_Cstream_k_E_LT_weight, init_Cstream_k_R_LT_weight, init_Cstream_v_LT_weight,
#                                                                            init_Cstream_segment_same_weight, init_Cstream_segment_notsame_weight, init_Cstream_segment_bias, init_Cstream_output_weight}
#

#operation : q_LT, k_LT, v_LT, segmentation, permutation(optional), masking, relative_position_encoding, segment_encoding, cal_content_attention, cal_query_attention, memorise

#properties : Qstream_q_LT_weight, Qstream_q_K_weight,Qstream_q_R_weight, Qstream_k_E_LT_weight, Qstream_k_R_LT_weight, Qstream_v_LT_weight,
#             Qstream_segment_same_weight, Qstream_segment_notsame_weight, Qstream_segment_bias, Qstream_output_weight,
#             Cstream_q_LT_weight, Cstream_q_K_weight,Cstream_q_R_weight, Cstream_k_E_LT_weight, Cstream_k_R_LT_weight, Cstream_v_LT_weight,
#             Cstream_segment_same_weight, Cstream_segment_notsame_weight, Cstream_segment_bias, Cstream_output_weight content_attention, query_attention, content_memory, query_memory

#output : Dq vector(s)



In [ ]:
class create_PLM_input(data_utils.Dataset):
  def __init__(self, data, target_csv, word_dict = False, sep_by_lines = False, segment_len = False, do_shuffle=True):
    super(create_PLM_input, self).__init__()
    
    

In [ ]:
class create_PLM_input_inmemory(data_utils.Dataset):


In [ ]:
# https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html

perred


In [ ]:
a= torch.nn

In [ ]:
def shuffle()

def permutate()

def rel_position_encoding():

def masking():

In [ ]:
if not a:
  print(1)

In [ ]:
class Two_stream_attention(nn):
  def __init__(self, q_size, k_size, v_size, attention_size = False, heads =8, **kwargs):
    super(Two_stream_attention, self).__init__() 
      

    self.segment_index = segment_index

    #initialize params
    self.initialize_params(self, q_size, k_size, v_size, attention_size, heads)

  def initialize_params(self, q_size, k_size, v_size, attention_size, heads):
    self.Qstream_q_LT_weight = nn.Linear()
    self.Qstream_q_K_weight = 
    self.Qstream_q_R_weight = 
    
    self.Qstream_k_E_LT_weight = 
    self.Qstream_k_R_LT_weight = 
    self.Qstream_v_LT_weight = 
    
    self.Qstream_segment_same_weight = 
    self.Qstream_segment_notsame_weight = 
    self.Qstream_segment_bias = 
    
    self.Qstream_output_weight = 

    self.Cstream_q_LT_weight = 
    self.Cstream_q_K_weight = 
    self.Cstream_q_R_weight = 
    
    self.Cstream_k_E_LT_weight = 
    self.Cstream_k_R_LT_weight = 
    self.Cstream_v_LT_weight = 

    self.Cstream_segment_same_weight = 
    self.Cstream_segment_notsame_weight = 
    self.Cstream_segment_bias = 
    
    self.Cstream_output_weight =

  def set_params(self,**kwargs):
    for i,j in kwargs.items():
      setattr(self, i, j)
      explicitly_defined_vars.append(i)

  def forward(self, input, do_parm = False, rel_position = False):
    
    #record position
    if not rel_position:
      self.rel_position = rel_position_encoding(input)
    else:
      self.rel_position = rel_position

    #permutation
    if do_parm:
      self.permutate()

    #get position encoding

    #cal
  def backward(self,??): #클레스 정의하는 법 참조


SyntaxError: ignored

In [ ]:
_#http://nlp.seas.harvard.edu/2018/04/03/attention.html

In [ ]:
##Copyright 2013, Youngbin Jang, All rights reserved.